In [1]:
from memory import MemoryBuffer
from policies import PPOTrainer
from utils import gaussian_likelihood, reshape_train_var
import tensorflow as tf
from tensorflow.layers import dense, conv2d, max_pooling2d, flatten
import numpy as np
import time
import gym
from env import CartPoleEnv

In [2]:
# env = CartPoleEnv()
env = gym.make('BipedalWalker-v2')

obs = tf.placeholder(tf.float32, shape=[None]+list(env.observation_space.shape))
dense1 = dense(obs, 32, activation=tf.tanh)
dense2 = dense(dense1, 32, activation=tf.tanh)
act_probs = dense(dense2, env.action_space.shape[0])

v_dense1 = dense(obs, 32, activation=tf.tanh)
v_dense2 = dense(v_dense1, 32, activation=tf.tanh)
value = dense(v_dense2, 1)

network = PPOTrainer(obs, act_probs, value, act_type='c')

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
n_episodes = 1000000
max_steps = 150
update_freq = 16
print_freq = 4

mb = MemoryBuffer(True)

In [8]:
all_rewards = []

for episode in range(n_episodes):
    ep_reward = 0
    
    mb.start_rollout()
    obs = env.reset()
    for step in range(max_steps):
        obs = obs.squeeze()
        obs = obs / 5.
        for i in obs:
            if abs(i) > 1:
                print('SDHFYUSDHFIUSDHF')
                print(i)
        act = network.gen_act(obs)
        print(obs)
        print(act)
        act = np.clip(act, -1, 1)
        obs_next, rew, d, _ = env.step(act)
        ep_reward += rew
        
        if False:
            env.render()
            time.sleep(0.02)
        
        mb.record(obs, act, rew, obs_next)
        obs = obs_next
        
        if d:
            break
            
    all_rewards.append(ep_reward)
            
    if episode % update_freq == 0 and episode != 0:
        network.train(mb.to_data())
        
        if episode % (update_freq * print_freq) == 0:
            print(f'Update #{episode // update_freq}, Reward: {np.mean(all_rewards[-update_freq*print_freq:])}')
            print()

[ 5.49496431e-04 -1.38731895e-06  1.07921083e-04 -3.19999075e-03
  1.84107915e-02 -1.42420223e-04  1.72043145e-01  3.85880346e-04
  2.00000000e-01  6.49189726e-03 -1.42409408e-04  1.70752960e-01
  1.00720099e-04  2.00000000e-01  8.81628036e-02  8.91640246e-02
  9.22845542e-02  9.79100406e-02  1.06820560e-01  1.20492208e-01
  1.41829777e-01  1.77186370e-01  2.00000000e-01  2.00000000e-01]
[nan nan nan nan]
[nan nan nan nan]


AssertionError: r.LengthSquared() > 0.0f